In [ ]:
import os
import time
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import networkx as nx
import yaml
import json
from scipy.stats.contingency import crosstab
import scipy
from scipy.cluster.hierarchy import linkage, dendrogram
import pickle

sys.path.append(os.getcwd())
sys.path.append((os.path.expanduser('~')))
#import functions.visualise as vis
import PpaPy.plot2 as vis
import functions.process as proc
from functions.io import setup_logger, makedir

from sklearn.cluster import AgglomerativeClustering

In [ ]:
def load_data_from_keys(json_file, key):
    def traverse_dict(d, key):
        if isinstance(d, dict):
            if key in d.keys():
                return {key: d[key]}
            else:
                return {k: traverse_dict(v, key) for k, v in d.items()}
        elif isinstance(d, list):
            return [traverse_dict(x, key) for x in d]
        else:
            return d

    with open(json_file, 'r') as f:
        data = json.load(f)
    nested_dict = traverse_dict(data, key)
    return {(innerKey, outerKey): values for outerKey, innerDict in nested_dict.items() for innerKey, values in innerDict.items()} 

In [ ]:
def sig_stars(p):
    if p <= .01:
        return int(-np.ceil(np.log10(abs(p)))) # should be correct needs checking
    elif p <= .05:
        return 1
    else:
        return 0

Please provide where your files are stored and where you would like your data to be saved in the following section.

In [ ]:
datestr = time.strftime("%Y%m%d-%Hh%Mm")
inpath = "/gpfs/soma_fs/home/boeger/PpaPred/ErenBoeger_2024" 
#inpath = '/gpfs/soma_fs/home/boeger/PpaPred/PpaPred_roca_35727184'

outpath = inpath #"/gpfs/soma_fs/home/boeger/PpaPred/data_eren_26825048"

config_path = "config.yml"

# Eren Boeger 2024
include, fig = ["Exp1_WT_larvae", "Exp1_WT_OP50"], "Exp1"   
#include, fig = ["Exp2_WT_larvae", "Exp2_tdc1_larvae", "Exp2_tbh1_larvae"], "Exp2"
#include, fig = ["Exp3_ser3_larvae", "Exp3_ser6_larvae", "Exp3_lgc55_larvae"], "Exp3"
#include, fig = ["Supp5_WT_larvae", "Supp5_tyramine_larvae", "Supp5_octopamine_larvae"], "Supp5"
#include, fig = ["Exp2_tph1_larvae", "Exp2_cat2_larvae", "Supp4_tbh1tdc1_larvae"], "Supp4"
#include, fig = [ "Exp3_octr1_larvae", "Exp3_tyra2_larvae", "Exp3_ser2_larvae", "Exp3_tyra3_larvae", "Supp7_ser2tyra2tyra3_larvae"], "Supp7"

#include, fig = ['Exp2_WT_larvae_0624','Exp2_tph1_larvae', 'Exp2_cat2_larvae', ,"Exp2_tbh1tdc1_larvae"], 'Supp4_Pharma' #transitions
#include, fig = ['Exp3_WT_larvae',"Exp3_tyra2_larvae", "Exp3_ser2_larvae", "Exp3_tyra3_larvae","Exp3_tyra2tyra3ser2_larvae"], 'Supp7_Pharma' # transitions

file_pattern = 'batch.json'
inpath_with_subfolders = True

Z_path = False #os.path.join(inpath,f'Exp1_WT_larvae_linkage_other_norm_out.pkl')
plot = False

In [ ]:
#all_transitions = None
#all_durations = None

### I/O ################################################
all_files = [os.path.join(root, name) for root, dirs, files in os.walk(inpath) for name in files if file_pattern in name and any([c in name for c in include]) and name.endswith('json')]
all_files.sort(key = lambda i: np.where([c in i for c in include])) # to sort all_files as given in include
loc_all = {os.path.basename(f):f for f in all_files}

outpath = makedir(os.path.join(inpath,fig))
    
### Configuration ################################################
config = yaml.safe_load(open(config_path, "r"))
cluster_color = config['cluster_color']
cluster_group = config['cluster_labels']
cluster_label = config['cluster_labels']
clu_group_label = {_:f'{_}, {__}' for _, __ in tuple(zip([c for c in cluster_label.values()],[g for g in cluster_group.values()]))}
skip_already = config['settings']['skip_already']
fps = 30
loc_all

In [ ]:
#####
def transplot(fr_transition_norm, ordering=None, cmap='viridis', vmin=0, vmax=1, linked=None, label='[]'):
    fig = plt.figure()
    
    if ordering is None:
        ordering = list(range(len(fr_transition_norm)))
    
    if linked is not None:
        axs1 = fig.add_axes([0, .895, .2, .805])
        axs1.axis('off')
        dn = dendrogram(linked, orientation='left',ax= axs1, color_threshold=0,above_threshold_color='k')
        
    axs2 = fig.add_axes([0.35, .9, .8, .8])    
    im = axs2.imshow(fr_transition_norm.iloc[ordering,ordering], cmap =cmap, vmin=vmin, vmax=vmax) # x state i y state i+1
    axs2.set_xticks(range(len(ordering)))
    axs2.set_xticklabels([cluster_group[k] for k in ordering], rotation=45,ha="center")
    axs2.set_xlabel('State i')
    axs2.set_yticks(range(len(ordering)))
    axs2.set_yticklabels([cluster_group[k] for k in ordering])
    axs2.set_ylabel('State i+1')
    cbar = axs2.figure.colorbar(im, ax=axs2)
    #cbar.ax.set_ylabel("X^0.4 normalization", rotation=90, labelpad= 6)
    axs2.set_title(f"{label}")
    #axs2.set_title(f"transitions per sec\n{label}")
    return fig

In [ ]:
inpath

In [ ]:
metric = 'mean transitions'
outmetric = os.path.join(outpath, f"{datestr}_{''.join(metric.split(' '))}_Utest")

metric_multi = None
for fn, fpath in loc_all.items():
    batch_metric = pd.DataFrame(load_data_from_keys(fpath, metric)).droplevel(0, axis=1)
    batch_metric.columns = pd.MultiIndex.from_product([[fn], batch_metric.columns])
    if metric_multi is None:
        metric_multi = batch_metric
    else:
        metric_multi = pd.concat([metric_multi, batch_metric], axis=1)
    #batch_etho = load_data_from_keys(fpath, 'ethogram')
if metric == 'mean duration':
    metric_multi = metric_multi/fps
metric_multi.index.name = 'cluster'

all_conds = metric_multi.columns.get_level_values(0).unique()
id_cond = [c.split('_')[0] for c in all_conds]
metric_multi.index = pd.MultiIndex.from_tuples([eval(i) for i in metric_multi.index])
metric_multi = metric_multi.drop(-1, level = 0)
metric_multi = metric_multi.drop(-1, level = 1)


all_trans = {}
trans_suffix = '_other_norm_out'
for i, cond in enumerate(all_conds):
    fr_transition = metric_multi[cond].mean(axis=1).unstack(level=0).fillna(0)
    fr_transition_norm = (fr_transition/fr_transition.sum(axis=0))

    trans_self = np.diagonal(fr_transition.values)
    trans_other = fr_transition.copy()
    np.fill_diagonal(trans_other.values, 0)

    all_trans[cond] = fr_transition
    all_trans[cond+'_norm_out'] = (fr_transition/fr_transition.sum(axis=0))
    all_trans[cond+'_norm_in'] = (fr_transition.T/fr_transition.sum(axis=1)).T
    all_trans[cond+'_self'] = trans_self
    all_trans[cond+'_other'] = trans_other
    all_trans[cond+'_self_norm'] =  trans_self/trans_self.sum()
    all_trans[cond+'_other_norm_out'] = trans_other/trans_other.sum(axis=0)
    all_trans[cond+'_other_norm_in'] = (trans_other.T/trans_other.sum(axis=1)).T
    all_trans[cond+'_other_norm_inout'] = pd.DataFrame(np.triu(trans_other)/np.sum(np.triu(trans_other))+np.tril(trans_other)/np.sum(np.tril(trans_other)))
    
    #
    #np.round(all_trans[cond+trans_suffix],4).to_csv(os.path.join(inpath, fig, f'{cond.replace("_batch.json","")}_transitions{trans_suffix}.csv'))

In [ ]:
baseline_trans

In [ ]:
if Z_path == False:
    # trans_suffix is other_normed_out
    baseline_trans = all_trans[include[0]+'_batch.json'+trans_suffix].copy()
    
    method_str = 'DistSymNormSum'
    baseline_dist = (1/((baseline_trans+np.triu(baseline_trans).T+np.tril(baseline_trans).T)))
    np.fill_diagonal(baseline_dist.values,0)
    baseline_dist_norm = baseline_dist/sum(baseline_dist)
    
    baseline_dist_sq = scipy.spatial.distance.squareform(baseline_dist_norm)
    Z = linkage(baseline_dist_sq, 'single', optimal_ordering=True)#ward #complete
    
    # PLots
    ordering = dendrogram(Z)['leaves'][::-1]
    plt.show()
    
    dist_plot = transplot(baseline_dist_norm, ordering, linked=Z, label='Distance: 1/((transition_out + transitions_in)/sum(transitions)')#linked=Z,     plt.savefig(os.path.join(inpath,fig,f'{include[0]}_{method_str}_distancematrix_oneminus.pdf'), bbox_inches='tight')
    plt.show()
    
    org_plot = transplot(baseline_trans, ordering, linked=Z, label='original transitions')#linked=Z,
    #plt.savefig(os.path.join(inpath,fig,f'{include[0]}_{method_str}_transitionmatrix_oneminus.pdf'), bbox_inches='tight')
    plt.show()

    # Save
    np.round(baseline_dist_norm,4).to_csv(os.path.join(inpath, fig, f'{include[0]}_distance_other_normTotal.csv'))
    #with open(os.path.join(inpath, fig, f'{include[0]}_linkage{trans_suffix}.pkl'), 'wb') as f:  # open a text file
        #pickle.dump(Z, f) # serialize the list

elif isinstance(Z_path,str):
    with open(Z_path, 'rb') as f:  # open a text file
        Z = pickle.load(f)
        ordering = dendrogram(Z)['leaves'][::-1]
        plt.show()

In [ ]:
outmetric = os.path.join(outpath, f"{datestr}_{''.join('rel time in'.split(' '))}_Utest")

metric_multi_reltime = None
for fn, fpath in loc_all.items():
    batch_metric = pd.DataFrame(load_data_from_keys(fpath, 'rel time in')).droplevel(0, axis=1)
    batch_metric.columns = pd.MultiIndex.from_product([[fn], batch_metric.columns])
    if metric_multi_reltime is None:
        metric_multi_reltime = batch_metric
    else:
        metric_multi_reltime = pd.concat([metric_multi_reltime, batch_metric], axis=1)
    #batch_etho = load_data_from_keys(fpath, 'ethogram')
metric_multi_reltime.index.name = 'cluster'
nodes_alpha = metric_multi_reltime.groupby(level=0, axis=1).mean()


In [ ]:
trans_rel*100, all_trans[cond+trans_suffix]*100, .1*100

In [ ]:
for i, cond in enumerate(all_conds):
    transplot_cond = transplot(all_trans[cond+trans_suffix], ordering, linked=Z, label=cond+trans_suffix)# vmax=.3)#linked=Z, 
    plt.savefig(os.path.join(inpath,fig,f'{cond.split(".")[0]}_transitions{trans_suffix}.pdf'),  bbox_inches="tight")

    trans_rel = (all_trans[cond+trans_suffix]-all_trans[include[0]+'_batch.json'+trans_suffix])*metric_multi_reltime[cond].mean(axis=1).values
    transplot_diff = transplot(trans_rel, ordering,  cmap='RdBu_r',linked=Z, vmin=-.1,vmax=.1, label=cond+trans_suffix+' diff WT larvae')#linked=Z, 
    plt.savefig(os.path.join(inpath,fig,f'{cond.split(".")[0]}_transitions_relto_WTlarvae{trans_suffix}.pdf'),  bbox_inches="tight")

In [ ]:
for i, cond in enumerate(all_conds):
    
    mean_metric = metric_multi_reltime[cond].mean(axis=1)
    trans_ = all_trans[cond+trans_suffix]
    trans_self = all_trans[cond+'_self_norm']
    
    trans_other_alpha = trans_.T.values.flatten()
    trans_other_alpha = trans_other_alpha[trans_other_alpha>0]
    trans_other_alpha[trans_other_alpha >= .2] = 1
    trans_other_alpha[trans_other_alpha < .25] = .25
    
    label = (((mean_metric*100).round().astype(int)).astype(str)+ '%').to_dict()
    transition_plot = vis.transition_plotter(trans_.values.copy(), cluster_color,#trans_self.copy(), 
                                             node_alpha=nodes_alpha[cond], edge_alpha=trans_other_alpha.T, clu_group_label=label)
    plt.text(1.5, -1, f'{cond}\nN = {len(metric_multi_reltime[cond].T)}', fontsize=12)
    plt.title(f"transitions of {cond}")
    #"plt.savefig(os.path.join(inpath,fig,f'{cond.split(".")[0]}_transdiagramm{trans_suffix}.pdf'),  bbox_inches="tight")
    plt.show()
        

In [ ]:
all_trans[cond+'_other'], pd.DataFrame(np.triu(all_trans[cond+'_other'])/np.sum(np.triu(all_trans[cond+'_other']))+np.tril(all_trans[cond+'_other'])/np.sum(np.tril(all_trans[cond+'_other']))), np.sum(np.triu(all_trans[cond+'_other']))

In [ ]:
all_trans[cond+'_other_norm_inout'], np.sum(np.triu(all_trans[cond+'_other_norm_inout'])), np.sum(np.tril(all_trans[cond+'_other_norm_inout']))

In [ ]:
all_trans_single = {}
for i, cond in enumerate(all_conds):
    firstcol = metric_multi[cond].columns[1]
    fr_transition = metric_multi[(cond,firstcol)].unstack(level=0).fillna(0)
    fr_transition_norm = (fr_transition/fr_transition.sum(axis=0))

    trans_self = np.diagonal(fr_transition.values)
    trans_other = fr_transition.copy()
    np.fill_diagonal(trans_other.values, 0)

    all_trans_single[cond] = fr_transition
    all_trans_single[cond+'_norm_out'] = (fr_transition/fr_transition.sum(axis=0))
    all_trans_single[cond+'_norm_in'] = (fr_transition/fr_transition.sum(axis=1))
    all_trans_single[cond+'_self'] = trans_self
    all_trans_single[cond+'_other'] = trans_other
    all_trans_single[cond+'_self_norm'] =  trans_self/trans_self.sum()
    all_trans_single[cond+'_other_norm_out'] = trans_other/trans_other.sum(axis=0)
    all_trans_single[cond+'_other_norm_in'] = trans_other/trans_other.sum(axis=1)

    transplot_cond_single = transplot(all_trans_single[cond+trans_suffix], ordering, linked=Z, label=firstcol)
    #plt.savefig(os.path.join(inpath,fig,f'{cond.split(".")[0]}_transitions.pdf'),  bbox_inches="tight")

In [ ]:
np.fill_diagonal(trans_.values, 0)

In [ ]:
trans_

In [ ]:
dendrogram(Z)['leaves']

In [ ]:
stats_json = {'meta':{'metric': metric,
                          'statistical test': 'Mann-Whitney-U-Test, two-sided',
                          'bonferroni correction': 'statsmodels...multipletests',
                          'population': stat_pop},
              'adjusted ps': all_ps.to_dict(),
              'Mann-Whitney U': all_U1s.to_dict(),
              'unadjusted ps': all_ps_orig.to_dict(),
              }
jsnF = json.dumps(stats_json, indent = 4)
with open(os.path.join(outpath, f'{datestr}_{metric}_Utest.json'), "w") as outfile:
    outfile.write(jsnF)

In [ ]:
li = [eval(i) for i in metric_multi.index]
type(li[0])

In [ ]:
op.itemgetter('metric', 'statistical test', 'bonferroni correction', 'population')(stats_json)
op.getitem(stats_json,'metric')

In [ ]:
from scipy import stats
U1s, pvs = stats.mannwhitneyu(metric_multi[stat_pop], metric_multi['bac_data_batch.json'], axis=1, nan_policy='omit')

In [ ]:

___,ps_adjusted,__,_ = multipletests(pvs, method='bonferroni')
np.round(ps_adjusted,5)

In [ ]:


    ### Load data ################################################
    #data_batch = load_tolist(fpath, droplabelcol=False)
    #data_batch_concat = pd.concat([d for d in data_batch], axis=0)
    
    #y_batch_concat = data_batch_concat['prediction']
    #y_batch = [d['prediction'] for d in data_batch]

    
    ### Batch summary ############################################
    for i,d in enumerate(loc_summ):
        summ_ = pd.read_csv(d)
        dev_ = (~summ_.isna()).astype(int)
        summ_ = summ_.fillna(0)
        #trans_col_ = [[c for c in fr_transition_],[c for c in fr_transition_]]
        
        if i == 0:
            summ = summ_
            devider = dev_
        else:
            summ += summ_
            devider += dev_
    summary = summ/devider
    #summary.to_csv(os.path.join(outpath, f'{data_str}_batch_summary.csv'))
    
    total_dur = summary.duration_mean*summary.duration_count
    total_dur_rel = total_dur/total_dur.sum()
    
    ### Batch transitions ############################################
    #batch_transitions = np.full((len(loc_trans),9,9), np.nan) # with shape recordings,cluster from, cluster to        
    for i,d in enumerate(loc_trans):
        fr_transition_ = pd.read_csv(d, index_col=0)
        #batch_transitions[i] = fr_transition_
        trans_col_ = [[c for c in fr_transition_],[c for c in fr_transition_]]
        if i == 0:
            fr_transition = fr_transition_
            trans_col = trans_col_
        elif trans_col_ == trans_col:
            #print('this')
            fr_transition += fr_transition_
        else:
            print('WARNING')

    # merge all transitions across animals and experiments
    #if all_transitions is None:
    #    all_transitions = np.array([batch_transitions])
    #else:
    #    all_transitions,batch_transitions = padtoequalsize(all_transitions, batch_transitions, [-3])
    #     all_transitions = np.concatenate([all_transitions,[batch_transitions]],axis=0)

    # calculate the normalised transition and cluster
    # cluster behaviors with single linkage
    Z = linkage(fr_transition, 'single', optimal_ordering=True)
    # extract closest cluster
    ordering = np.concatenate((Z[::-1,0],Z[:,1]))
    ordering = ordering[ordering<len(cluster_label)]-1
    # calculate normalised transitions
    fr_trans_norm = (fr_transition/fr_transition.sum(axis=0)).fillna(0)
    fr_transition_clust = fr_trans_norm.iloc[ordering,ordering]
    fr_trans_norm.to_csv(os.path.join(outpath, f'{data_str}_batch_transitions_2.csv'), index=False)
    



    ### Batch durations ############################################
    #batch_durations = None
    for rec in loc_onoff:
        with open(rec,'r') as jf:
            onoff = json.load(jf)
        #duration = pd.DataFrame(dtype='float64')
        for cluster in onoff: #for bout in cluster
            cluster_dur = []
            for bout in onoff[cluster]: #for bout in cluster
                cluster_dur.append(bout[1])
            #duration = pd.concat([duration,pd.DataFrame([cluster_dur],index=[eval(cluster)],dtype='float64')],axis=0)
            
        cl_present = duration.index
        cl_range = range(min(cl_present), max(cl_present)+1)
        duration = duration.reindex(cl_range)
        
        if batch_durations is None:
            batch_durations = np.array([duration.values])
        else:
            batch_durations, duration = padtoequalsize(batch_durations, duration, [-1,-2])
            batch_durations = np.concatenate([batch_durations,[duration]],axis=0)
    
    if all_durations is None:
        all_durations = np.array([batch_durations])
    else:
        all_durations,batch_durations = padtoequalsize(all_durations, batch_durations, [-1,-2,-3])
        all_durations = np.concatenate([all_durations,[batch_durations]],axis=0)
    
        
    #############
    ### plots ###
    #############
    
    if plot:
        """
        transition_plot = vis.transition_plotter(fr_trans_norm.values, cluster_color, node_alpha=summary['duration_relative'].fillna(0).tolist())
        plt.text(1.5, -1, f'{data_str}\nN = {len(loc_trans)}', fontsize=12)
        plt.title(f"transitions of {data_str}")
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_transitions.pdf'))
        plt.show()"""
        
        fig = plt.figure()
        axs1 = fig.add_axes([0, .895, .2, .805])
        axs1.axis('off')
        dn = dendrogram(Z, orientation='left',ax= axs1)
        axs2 = fig.add_axes([0.33, .9, .8, .8])
        im = axs2.imshow(fr_transition_clust,norm=colors.PowerNorm(.4,vmax=1))#vmin=0,vmax=.2)#
        axs2.set_xticks(range(len(cluster_group)))
        axs2.set_xticklabels([cluster_group[k] for k in ordering], rotation=45,ha="center")
        axs2.set_yticks(range(len(cluster_group)))
        axs2.set_yticklabels([cluster_group[k] for k in ordering])
        cbar = axs2.figure.colorbar(im, ax=axs2)
        cbar.ax.set_ylabel("X^0.4 normalization", rotation=90, labelpad= 6)
        axs2.set_title(f"transitions of {data_str} per sec (x to y)")
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_transitheatmap_clust.pdf'),bbox_inches = "tight")
        plt.show()
    
        if WT_ordering:
            fr_transition_WT = fr_trans_norm.iloc[WT_ordering,WT_ordering]
            fig = plt.figure()
            axs1 = fig.add_axes([0, .895, .2, .805])
            axs1.axis('off')
            #dn = dendrogram(Z, orientation='left',ax= axs1)
            axs2 = fig.add_axes([0.33, .9, .8, .8])
            im = axs2.imshow(fr_transition_WT,norm=colors.PowerNorm(.4,vmax=1))#vmin=0,vmax=.2)#
            axs2.set_xticks(range(len(cluster_group)))
            axs2.set_xticklabels([cluster_group[k] for k in WT_ordering], rotation=45,ha="center")
            axs2.set_yticks(range(len(cluster_group)))
            axs2.set_yticklabels([cluster_group[k] for k in WT_ordering])
            cbar = axs2.figure.colorbar(im, ax=axs2)
            cbar.ax.set_ylabel("X^0.4 normalization", rotation=90, labelpad= 6)
            axs2.set_title(f"transitions of {data_str} per sec (x to y)")
            plt.savefig(os.path.join(outpath, f'{data_str}_batch_transitheatmap_WTclust.pdf'),bbox_inches = "tight")
            plt.show()
        
    
        
        fig, ax = plt.subplots(figsize=(2,5))
        b = ax.barh(range(len(total_dur_rel)),total_dur_rel, color=list(cluster_color.values())[1:])
        ax.set_yticks(range(len(cluster_group)-1))
        ax.set_yticklabels([cluster_group[k] for k in cluster_group][1:])
        ax.bar_label(b, label_type='edge', fmt='%.2g', padding=3)
        ax.invert_yaxis()
        plt.xlabel(f"total rel. duration")
        plt.title(data_str)
        plt.xlim(0,.6)
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_totaldur.pdf'),bbox_inches = "tight")
        plt.show()
    
        # Calculate onoff sets
        onoff = proc.onoff_dict(y_batch, labels =np.unique(y_batch_concat))
    
        fig = plt.figure(figsize=(.6*len(onoff.keys()),3), layout='constrained')
        for k in onoff.keys():
            if not np.isnan(k):
                durs = pd.DataFrame(onoff[k])[1]/30
                plt.boxplot(durs, positions=[k], widths=.5,sym='', patch_artist = True, boxprops={'facecolor':cluster_color[k]},medianprops={'color':'k'})
        plt.xticks(range(len(cluster_group)-1),[cluster_group[k] for k in cluster_group][1:], rotation=45)
        plt.ylabel("sec")
        plt.ylim(0,30)
        plt.title(f"duration of {data_str} (without fliers)")
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_durboxplot.pdf'),bbox_inches = "tight")
        plt.show()

In [ ]:
all_durations.shape

In [ ]:
cl_present

In [ ]:
def df_withnans_alignright(df_):
    df = df_.copy()
    for i in range(len(df.columns)):
        nulls = df.iloc[np.where(df.iloc[:,i].notnull())[0][-1]+1:,i]
        notnulls = df.iloc[:np.where(df.iloc[:,i].notnull())[0][-1],i]
        right_aligned = pd.concat([nulls,notnulls], axis=0).reset_index(drop=True)
        df.iloc[:,i] = right_aligned
    return df

In [ ]:
np.random.seed(96)
bout_all = pd.DataFrame([])
bout_pre_all = pd.DataFrame([])
y_pre_all = pd.DataFrame([])
bout_post_all = pd.DataFrame([])
y_post_all = pd.DataFrame([])
feat = 'velocity'
cl = 1
rdm_subset = np.random.choice(len(onoff[cl]), 100, replace=False)
rdm_ons =[]
for i,oo in enumerate(rdm_subset):
    rdm_ons.append(onoff[cl][oo][0])
    onset = onoff[cl][oo][0]
    offset = onoff[cl][oo][0]+onoff[cl][oo][1]
    bout = data_batch_concat.reset_index(drop=True)[onset:offset].reset_index(drop=True)
    if all(bout[feat].isnull()):
        print(i)
        continue
    bout_pre = data_batch_concat.reset_index(drop=True)[onset-60:onset].reset_index(drop=True)
    bout_post = data_batch_concat.reset_index(drop=True)[offset:offset+60].reset_index(drop=True)
    
    bout_all = pd.concat([bout_all,bout[feat]], axis=1)

    bout_pre_all = pd.concat([bout_pre_all,bout_pre[feat]], axis=1)
    y_pre_all = pd.concat([y_pre_all,bout_pre['prediction']], axis=1)
    bout_post_all = pd.concat([bout_post_all,bout_post[feat]], axis=1)
    y_post_all = pd.concat([y_post_all,bout_post['prediction']], axis=1)

bout_all.columns = list(range(len(bout_all.columns)))
bout_pre_all.columns = list(range(len(bout_pre_all.columns)))
y_pre_all.columns = list(range(len(y_pre_all.columns)))
y_pre_color = y_pre_all[:150].replace(cluster_color)
bout_post_all.columns = list(range(len(bout_post_all.columns)))
y_post_all.columns = list(range(len(y_post_all.columns)))
y_post_color = y_post_all[:150].replace(cluster_color)

bout_all_right = df_withnans_alignright(bout_all)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
mask = data_batch_concat.prediction.isin([7])
#fig = px.lines(bout_all, color = y_bout_all, width=800, height=400)
fig = make_subplots(rows=1, cols=2)
for trace in bout_all:
    bout_ = bout_all[trace].iloc[:90].dropna().reset_index(drop=True)
    bout_r_ = bout_all_right[trace].iloc[-90:].dropna().reset_index(drop=True)
    pre_ = bout_pre_all[trace][y_pre_all[trace]==y_pre_all[trace].iloc[-1]].reset_index(drop=True)
    post_ = bout_post_all[trace][y_post_all[trace]==y_post_all[trace].iloc[-1]].reset_index(drop=True)
    
    fig.add_trace(go.Scatter(x=np.arange(len(bout_)), y=bout_, line=dict(color=cluster_color[cl]),mode='lines'),
                 row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(-len(pre_),0), y=pre_, line=dict(color=y_pre_color[trace].iloc[-1]),mode='lines'),
                 row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(-len(bout_r_),0), y=bout_r_, line=dict(color=cluster_color[cl]),mode='lines'),
                 row=1, col=2)
    fig.add_trace(go.Scatter(x=np.arange(len(post_)), y=post_, line=dict(color=y_post_color[trace].iloc[-1]),mode='lines'),
                 row=1, col=2)

    
mean_ = pd.concat([bout_pre_all,bout_all],axis=0).reset_index(drop=True).mean(axis=1).iloc[:150]
mean_r_ = pd.concat([bout_all_right, bout_post_all],axis=0).reset_index(drop=True).mean(axis=1).iloc[-150:]
fig.add_trace(go.Scatter(x=np.arange(-60,90), y=mean_, line=dict(color='black',width=3),mode='lines'),
                 row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(-90,60), y=mean_r_, line=dict(color='black',width=3),mode='lines'),
                 row=1, col=2)

fig.add_vline(x=0, line=dict(color="Grey",width=5))
fig.update_xaxes(dtick=15, title=dict(text='Time [frames]'))
fig.update_yaxes(range=[0, max(bout_all.max().max(),bout_pre_all.max().max(),bout_post_all.max().max())],title=dict(text=f'{feat}'))
fig.update_layout(
    width=1200,
    height=500,
    xaxis = dict(
        tickmode = 'linear',
        dtick = 15),
    showlegend=False,
    template='plotly_white',
    title=dict(text=f"Traces of {cluster_group[cl]} aligned at prediction onset (l) and offset (r)", font=dict(size=16), x =.5, xanchor='center'))
fig.show()

In [ ]:
pd.concat([bout_pre_all,bout_all], axis=0).reset_index(drop=True)[:150]

In [ ]:
onoff_ = pd.json_normalize(onoff).to_dict(orient='split')
onoff_rev = {}
for l,c in zip(onoff_['data'][0],onoff_['columns']):
    for oo in l:
        onoff_rev[oo[0]] = (c, oo[1])

onoff_rev_df = pd.DataFrame(onoff_rev).T
onoff_rev_df = onoff_rev_df.reindex(sorted(onoff_rev_df.index), axis=0)

onoff_rev_dfidx = onoff_rev_df.index.to_series().reset_index(drop=True)
prebout_idx = (onoff_rev_dfidx[onoff_rev_dfidx.isin(rdm_ons)].index - 1).tolist()

prebout_c, prebout_dur, prebout = onoff_rev_df.iloc[prebout_idx,0],onoff_rev_df.iloc[prebout_idx,1], onoff_rev_df.index[prebout_idx].to_list()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(8,5), sharey=True)
axs[0].plot(range(0,90),bout_all[:90], c=cluster_color[cl],alpha=.7)
for b,d,c in zip(prebout,prebout_dur,prebout_c):
    axs[0].plot(range(-d if d < 60 else -60,0),data_batch_concat.reset_index(drop=True).iloc[b:b+d][feat][-d if d < 60 else -60:], c=cluster_color[c],alpha=.7)
axs[0].plot(range(0,90),bout_all[:90].median(axis=1), c='#252422', linewidth=2)
axs[0].axvline(0)
axs[0].set_title("aligned beginning")
axs[1].plot(range(-90,0),bout_all_right[-90:], c=cluster_color[cl],alpha=.7)
axs[1].plot(range(-90,0),bout_all_right[-90:].median(axis=1), c='#252422', linewidth=2)
axs[1].axvline(0)
axs[1].set_title("aligned end")

In [ ]:
y_pre_all[trace][y_pre_all[trace]==y_pre_all[trace].iloc[-1]]

In [ ]:
import scipy
from scipy.cluster.hierarchy import linkage, dendrogram
Z = linkage(fr_transition, 'single', optimal_ordering=True)
ordering = np.concatenate((Z[::-1,0],Z[:,1]))
ordering = ordering[ordering<=8]
fr_trans_norm = (fr_transition/fr_transition.sum(axis=0)).fillna(0)
fr_transition_clust = fr_transition.iloc[ordering,ordering]
fr_trans_clust_norm = 

In [ ]:
fig = plt.figure()
axs1 = fig.add_axes([0, .895, .2, .805])
axs1.axis('off')
dn = dendrogram(Z, orientation='left',ax= axs1)
axs2 = fig.add_axes([0.33, .9, .8, .8])
im = axs2.imshow(fr_transition_clust,norm=colors.PowerNorm(.4,vmax=1))#vmin=0,vmax=.2)#
axs2.set_xticks(range(len(cluster_group)-1))
axs2.set_xticklabels([cluster_group[k] for k in ordering], rotation=45,ha="center")
axs2.set_yticks(range(len(cluster_group)-1))
axs2.set_yticklabels([cluster_group[k] for k in ordering])
cbar = axs2.figure.colorbar(im, ax=axs2)
cbar.ax.set_ylabel("X^0.4 normalization", rotation=90, labelpad= 6)
axs2.set_title(f"transitions of {data_str} per sec (x to y)")
plt.savefig(os.path.join(out_predicted, os.path.basename(outpath)+'_batch_transitheatmap_clust.pdf'),bbox_inches = "tight")
plt.show()

In [ ]:
fr_trans_norm

In [ ]:
fig, ax = plt.subplots(figsize=(2/.6*1,5))
b = ax.barh(range(len(total_dur_rel)),total_dur_rel, color=list(cluster_color.values())[1:])
ax.set_yticks(range(len(cluster_group)-1))
ax.set_yticklabels([cluster_group[k] for k in cluster_group][1:])
ax.bar_label(b, label_type='edge', fmt='%.2g', padding=3)
ax.invert_yaxis()
plt.xlabel(f"total rel. duration")
plt.title(data_str)
plt.xlim(0,1)
plt.savefig(os.path.join(out_predicted, os.path.basename(outpath)+'_batch_totaldur.pdf'),bbox_inches = "tight")
plt.show()

In [ ]:
    data_describe = data_batch_concat.groupby(y_batch_concat).describe().T.loc[idx[:, ['mean','std','count']], :].sort_index(level=0).T
    dur_describe = pd.DataFrame(dur, columns=['duration']).groupby(transi).describe().T.loc[idx[:, ['mean','std','count']], :].sort_index(level=0).T
    dur_describe['duration','relative'] = pd.DataFrame(dur, columns=['duration']).groupby(transi).apply(lambda cd: cd.sum()/len(y_batch_concat))
    summary = pd.concat([dur_describe, data_describe], axis=1)
    summary.index.name = 'cluster'
    summary = summary.T.reset_index(drop=True).set_index(summary.T.index.map('_'.join)).T
    summary = summary.set_index(summary.index.astype(int))
    summary = summary.reindex([k for k in cluster_label if k != -1])
    summary.to_csv(os.path.join(out_predicted, os.path.basename(outpath)+'_batch_summary.csv'))

    #### Older Version
    
    for i,d in enumerate(data_batch):
        frame = d['prediction'].rolling(30).apply(lambda s: s.mode()[0])[29::30].values.flatten()
        trans_col_,fr_transition_ = crosstab(frame[1:], frame[:-1], levels=([k for k in cluster_label if k != -1],[k for k in cluster_label if k != -1]))
        #fr_transition_ = pd.read_csv() ##################################read transitions.csv should look like normal fr_transition, get trans_col from header
        if i == 0:
            fr_transition = fr_transition_
            trans_col = trans_col_
        if trans_col_ == trans_col:
            fr_transition += fr_transition_
        else:
            print('WARNING')
        #fr_transition/fr_transition.sum(axis=0)
    
    #othersum_axis0 = fr_transition.sum(axis=0)-fr_transition.diagonal()
    #transition_toother = fr_transition/othersum_axis0
    #transition_self = fr_transition.diagonal()/(othersum_axis0+fr_transition.diagonal())
    #np.fill_diagonal(transition_toother, 0)

In [ ]:
pd.DataFrame(load_data_from_keys(fpath, 'ethogram')).droplevel(0, axis=1)

In [ ]:
load_data_from_keys(fpath, 'ethogram')